In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
cwd = os.getcwd()
path_train = os.path.join(cwd, 'dataset/train.csv')

In [ ]:
df_train = pd.read_csv(path_train)

In [ ]:
df_train

## Tratando valores faltantes

O primeiro passo para um bom resultado em um projeto de Aprendizado de Máquina é tratar valores faltantes. Então, a primeira coisa que fazemos aqui é remover os atributos que apresentam quantidade elevada de missing values, que são: 'MiscFeature', 'Fence', 'PoolQC', 'Alley', 'FireplaceQu'. Aplicar imputação nesse atributos geraria um viés muito grande no conjunto de dados devido a números exorbitantes de valores que não são apresentados, como por exemplo 1453 missing values no atributo 'PoolQC', logo a melhor solução é não utilizar tais atributos.

Após essa remoção, resta features com pequenas quantidades de valores faltantes, com o maior número sendo 81. Para essas features nós utilizamos técnicas de imputação. Em dados categóricos é utilizada a moda do atributo como valor para substituição, enquanto em dados de valor contínuo a média foi utilizada.

In [ ]:
df_train.isna().sum().values

In [ ]:
# Removendo os atributos com muitos dados faltantes
df_train = df_train.drop(['MiscFeature', 'Fence', 'PoolQC', 'Alley', 'FireplaceQu'], axis=1)

# Aplicando imputação
df_train['LotFrontage'] = df_train['LotFrontage'].fillna(df_train['LotFrontage'].mean())
df_train['BsmtQual'] = df_train['BsmtQual'].fillna(df_train['BsmtQual'].mode()[0])
df_train['BsmtCond'] = df_train['BsmtCond'].fillna(df_train['BsmtCond'].mode()[0])
df_train['BsmtExposure'] = df_train['BsmtExposure'].fillna(df_train['BsmtExposure'].mode()[0])
df_train['BsmtFinType1'] = df_train['BsmtFinType1'].fillna(df_train['BsmtFinType1'].mode()[0])
df_train['BsmtFinType2'] = df_train['BsmtFinType2'].fillna(df_train['BsmtFinType2'].mode()[0])
df_train['Electrical'] = df_train['Electrical'].fillna(df_train['Electrical'].mode()[0])
df_train['GarageType'] = df_train['GarageType'].fillna(df_train['GarageType'].mode()[0])
df_train['GarageYrBlt'] = df_train['GarageYrBlt'].fillna(df_train['GarageYrBlt'].mode()[0])
df_train['GarageFinish'] = df_train['GarageFinish'].fillna(df_train['GarageFinish'].mode()[0])
df_train['GarageQual'] = df_train['GarageQual'].fillna(df_train['GarageQual'].mode()[0])
df_train['GarageCond'] = df_train['GarageCond'].fillna(df_train['GarageCond'].mode()[0])
df_train['MasVnrType'] = df_train['MasVnrType'].fillna(df_train['MasVnrType'].mode()[0])
df_train['MasVnrArea'] = df_train['MasVnrArea'].fillna(df_train['MasVnrArea'].mean())

In [ ]:
df_train.isna().sum().values

## Tratando Features nominais

Para atributos nominais foi aplicada a numerização, que consiste em substituir as classes de um atributo por números. Esse passo é necessário para que o treino pelos modelos de aprendizado de máquina seja possível.

In [ ]:
def nom_to_num(column):
    """
    Recebe um atributo como parâmetro, transforma cada categoria em números e retorna uma Series do vetor transformado
    """

    # Extrai as diferentes classes do atributo 
    keys = column.value_counts().keys()
    i = 0

    # Para cada classe atribui um número
    for key in keys:
         column = column.replace(to_replace = key, value = i)
         i += 1

    return column

In [ ]:
def handle_nom(df, columns):
    """
    Recebe um DataFrame e os atributos a serem transformados de nominais para numéricos e retorna um DataFrame com esses atributos      transformados
    """

    # Para cada coluna, aplica a função de transformação
    for feature in columns:
        df[feature] = nom_to_num(df[feature])

    return df


In [ ]:
# Colunas com atributos nominais para serem transformados em númericos

columns=['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood', 'Condition1','Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']

In [ ]:
df_train_final = handle_nom(df_train, columns).drop('Id', axis=1)

In [ ]:
df_train_final

## Divisão dos dados

Para podermos analisar e melhorar os parâmetros utilizados no regressor, os dados serão separados em conjunto de treino e validação, sendo o conjunto de validação utilizado para fazermos um _tunning_ no modelo. Essa separação é necessária para verificarmos o tradeoff viés-variância antes de aplicarmos o modelo no conjunto de testes

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df_train_final.drop('SalePrice', axis=1)
y = df_train_final['SalePrice']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=9)

## Regressores


### Análise de modelos

    - Os modelos aqui analisados serão: RandomForestRegressor, SVM e uma rede neural

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

### Redução de dimensionalidade e Normalização

Nesta seção é aplicado o algoritmo de redução de dimensionalidade PCA. Ao utilizarmos algoritmos mais simples, como Random Forest e SVM devemos ter cuidado com a quantidade de features, e.g. dimensão, dos nossos dados, pois algumas características podem inserir ruído e acabar por "confundir" o modelo.

O PCA é um algoritmo não-supervisionado que analisa a variância entre cada feature e gera uma transformação linear de forma a maximizar essa variância. Com a variância maximizada, características que não acrescentam ao modelo, ou que prejudicam, são retiradas.

Para selecionar a quantidade de componentes do PCA que utilizaremos, passamos o valor .95, que indica que queremos o número de features que representam 95% da variância da base de dados. Dessa forma, são retornadas 54 features, que são utilizadas para o Rnadom Forest e para o SVM. Como as features geradas pelo PCA são uma combinação de outras características, elas não apresentam um real significa como a features originais.

Para a Rede Neural, devido ao método ser mais robusto e complexo, utilizaremos o conjunto de dados original, com 74 features.

Também é aplicada normalização dos dados do conjunto. a normalização é feita de tal forma que a distribuição de todas as características estejam entre 0 e 1. Como o conjunto apresenta uma variação bem grande na distribuição dos dados, a normalização acaba por facilitar a identificação de determinados padrões pelo modelo. A normalização é feita através da subtração da média e divisão pelo desvio padrão em cada feature separadamente



In [ ]:
# Normalização dos dados utilizando StandardScaler

std = StandardScaler()
std.fit(X_train)
X_train_std = std.transform(X_train)

In [ ]:
# Utilização do PCA nos dados normalizados

pca = PCA()
pca.fit(X_train_std)
sum_var = np.cumsum(pca.explained_variance_ratio_)*100

In [ ]:
# Plottando a variância explicada em relação à quantidade de componentes dos PCA

d = [n for n in range(len(sum_var))]
plt.figure(figsize=(10, 10))
plt.plot(d,sum_var, color = 'red',label='cumulative explained variance')
plt.ylabel('Cumulative Explained variance')
plt.xlabel('Principal components')
plt.axhline(y = 95, color='k', linestyle='--', label = '95% Explained Variance')
plt.legend(loc='best')

### Random Forest Regressor

O Random Forest Regressor é um algoritmo de ensemble que utiliza o método de Bagging. Funciona da seguinte forma: o conjunto de dados é separado em vários subconjuntos menores, e cada conjunto desse é independente, ou seja, não possuem dados sobressalentes. Então são contruídas várias árvores profundas e treinadas com diferentes subconjuntos de forma que cada árvore extraia diferentes conceitos de cada subconjunto. Então as árvores são agregadas e a média do resultado de todas as árvores é a saída final do Random Forest

Em geral, um conjunto de regressores fracos apresentam melhor resultado que um único regressor forte.

Foi utilizado o GridSearch Cross-Validation com 10 partições para identificar o conjunto de hiperparâmetros que tem a melhor performance, utilizando RMSE como métrica.


In [ ]:
# Aplicando Standardization e PCA aos dados
pca = PCA(.95)
pca.fit(X_train_std)
X_train_pca = pca.transform(X_train_std)

# Construindo um GridSearch para identificar os melhores hiperparâmetros para o regressador
grid_search = GridSearchCV(estimator=RandomForestRegressor(),param_grid={ 'max_depth': range(3,7),'n_estimators': (10, 50, 100, 500),},cv=10, scoring='neg_root_mean_squared_error', verbose=0, n_jobs=-1)

In [ ]:
# Executando a busca pelos melhores hiperparâmetros
grid_result = grid_search.fit(X_train_pca, y_train)

In [ ]:
# Resultado da busca
grid_result.best_params_

In [ ]:
# Criando o modelo utilizando os hiperparâmetros encontrados
rf_model = RandomForestRegressor(max_depth=grid_result.best_params_['max_depth'], n_estimators=grid_result.best_params_['n_estimators'], random_state=9)

In [ ]:
# Treinando o modelo
rf_model.fit(X_train_pca, y_train)

In [ ]:
# Resultado do modelo
y_pred = rf_model.predict(pca.transform(X_val))
rf_result = np.sqrt(mean_squared_error(y_val, y_pred))
print('RMSE = ', round(rf_result, 4))

### SVM

O SVM, usualmente muito utilizado para classificação, também pode ser utilizado para regressão. Os princípios do algorítmo continuam o mesmo: definir um hiperplano e maximizar a margem entre os dados. O algorítmo apresenta certa aceitação de erros, o que pode prejudicar sua performance em problemas de regressão, já que a saída é em intervalo contínuo e apresenta infinitas possibilidades. O SVM é um modelo muito robusto e na "era" pré Deep Learning era tão usado quanto as redes neurais atualmente, devido a sua capacidade de aprender diversas funções complexas.

Fazemos aqui novamente o uso do GridSearch para encontrarmos os melhores parâmetros possíveis dentro de um espaço de busca pré-definido.



In [ ]:
from sklearn import svm

In [ ]:
# Definindo o espaço de busca para o GridSearch

grid_search = GridSearchCV(svm.SVR(), param_grid={'kernel': ('poly', 'rbf', 'sigmoid'), 'degree': (2,3,4,5), 'gamma': np.linspace(0,1, num=5), 'C': (1,5,10,20,80,100, 200)})

In [ ]:
# Fazendo a busca pelos melhores hiperparâmetros

grid_result = grid_search.fit(X_train_pca, y_train)

In [ ]:
# Resultado da busca

grid_result.best_params_

In [ ]:
# Criando o modelo utilizando ps hiperparâmetros encontrados

svm_model = svm.SVR(kernel=grid_result.best_params_['kernel'], gamma=grid_result.best_params_['gamma'], degree=grid_result.best_params_['degree'], C=grid_result.best_params_['C'])

In [ ]:
# Treinando o modelo

svm_model.fit(X_train_pca, y_train)

In [ ]:
# Avaliação do modelo utilizando RMSE

y_pred = svm_model.predict(pca.transform(X_val))
svm_result = np.sqrt(mean_squared_error(y_val, y_pred))
print('RMSE = ', round(svm_result, 4))

### Rede Neural

Um dos maiores passos da computação nesse década, o Deep Learning apresenta robustez maior que os algorítmos de Aprendizagem clássicos apresentados anteriormente. Uma Rede Neural consiste basicamente da tentativa de encontrar o mínimo global do gradiente de uma função de perda. Ela é composta por um número $N$ de camadas e cada camada $N_i$ possui uma quantidade $X_i$ de neurônios e seus respectivos pesos. A rede neural recebe um vetor de features e o resultado baseado na função de ativação que cada camada recebeu. Após isso, é computado o erro utilizando uma função de perda e os pessoas da rede neural são atualizados.

Devido a complexidade e robustez do método, foi utilizado o banco de dados original, sem aplicação do PCA ou Normalização das características.

**O modelo criado para esse problema utiliza:**

    -   4 camadas:
        -   primeira camada: 30 neurônios; função de ativação: ReLU; Tamanho de entrada: 74 features;
        -   segunda camada: 15 neurônios; função de ativação: ReLU;
        -   terceira camada: 20 neurônios; função de ativação: ReLU;
        -   camada de saída: 1 neurônios; função de ativação: Linear.
    -   função de perda: Mean Squared Error;
    -   otimizador: adam;
    -   métricas utilizadas: Mean Absolute Error, Mean Squared Logarithmic Error;
    -   Épocas: 1000;
    -   Batch Size: 10;
    -   Conjunto de Validação: 20% do conjunto original.

In [ ]:
# Importando os métodos necessários do Keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# Função para criar o modelo

def create_model():
    NN = Sequential()
    NN.add(Dense(30, input_dim = 74, activation='relu'))
    NN.add(Dense(15, activation='relu'))
    NN.add(Dense(20, activation='relu'))
    NN.add(Dense(1))

    return NN

In [ ]:
# Criando o modelo

nn_model = create_model()
nn_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error', 'mean_squared_logarithmic_error'])

In [ ]:
# Treinando a rede neural

history = nn_model.fit(X, y, epochs=1000, batch_size=10, validation_split=0.2)

In [ ]:
# Métricas para avaliação do modelo

plt.figure(figsize=(15, 10))
plt.subplot(221)
plt.plot(np.sqrt(history.history['loss']))
plt.plot(np.sqrt(history.history['val_loss']))
plt.title('model RMSE')
plt.ylabel('RMSE')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='best')

plt.subplot(222)
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('model MAE')
plt.ylabel('MAE')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='best')

plt.subplot(223)
plt.plot(history.history['mean_squared_logarithmic_error'])
plt.plot(history.history['val_mean_squared_logarithmic_error'])
plt.title('model MSLE')
plt.ylabel('MSLE')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='best')

In [ ]:
nn_result = np.sqrt(history.history['loss'][-1])
print('RMSE Final: ', round(nn_result, 4))

## Comparação entre os resultados de cada modelo

Como tese para esse Desafio, busco apresentar uma comparação da capacidade de dois modelos clássicos de Aprendizado de Máquina e uma Rede Neural Densa e Superficial.

Para a Rede Neural buscou-se o melhor conjunto de hiperparâmetros de forma empírica e foram testados diferentes tratamento dos dados, como os utilizados para o SVM e o Random Forest. A configuração apresentada é a que mostrou melhor resultado em relação ao tradeoff viés-variância. Como foi separado 20% do conjunto para validação, podemos verificar que a diferença do erro de treino e o erro de validação são relativamente pequenos ,levando em consideração o intervalo do vetor 'SalePrice'. O intuito não foi buscar o melhor resultado possível no conjunto de treino, mas sim um equilíbrio entre o erro no conjunto de treino e um erro no conjunto de validação, para que o modelo tenha uma boa performance no conjunto de teste.

Já nos métodos clássicos, foi utilizado o GridSearch para fazer uma busca pelo melhores hiperparâmetros. Porém, a rede neural continua com um resultado muito melhor.

In [ ]:
print('RMSE Random Forest: ', round(rf_result, 4))
print('RMSE SVM: ', round(svm_result, 4))
print('RMSE Rede Neural: ', round(nn_result, 4))

Utilizando a raíz quadrada da média do erro ao quadrado (RMSE) podemos verificar a grande diferença entre os resultados obtidos pela rede neural e pelos algorítmos clássicos.

Por esse motivo foi decidido utilizar a rede neural construída aqui para fazer a regressão dos dados de teste e geral o resultado.

# Resultado no conjunto de teste

## Processamento dos dados de teste

O mesmo processamento aplicado nos dados de treino foi aplicado nos dados de teste, com uma pequena diferença nas features que sofreram imputação

In [ ]:
path_test = os.path.join(cwd, 'dataset/test.csv')

In [ ]:
df_test = pd.read_csv(path_test)

In [ ]:
df_test = df_test.drop(['MiscFeature', 'Fence', 'PoolQC', 'Alley', 'FireplaceQu'], axis=1)
df_test['LotFrontage'] = df_test['LotFrontage'].fillna(df_test['LotFrontage'].mean())
df_test['MSZoning'] = df_test['MSZoning'].fillna(df_test['MSZoning'].mode()[0])
df_test['Utilities'] = df_test['Utilities'].fillna(df_test['Utilities'].mode()[0])
df_test['Exterior1st'] = df_test['Exterior1st'].fillna(df_test['Exterior1st'].mode()[0])
df_test['Exterior2nd'] = df_test['Exterior2nd'].fillna(df_test['Exterior2nd'].mode()[0])
df_test['MasVnrType'] = df_test['MasVnrType'].fillna(df_test['MasVnrType'].mode()[0])
df_test['MasVnrArea'] = df_test['MasVnrArea'].fillna(df_test['MasVnrArea'].mean())
df_test['BsmtQual'] = df_test['BsmtQual'].fillna(df_test['BsmtQual'].mode()[0])
df_test['BsmtCond'] = df_test['BsmtCond'].fillna(df_test['BsmtCond'].mode()[0])
df_test['BsmtExposure'] = df_test['BsmtExposure'].fillna(df_test['BsmtExposure'].mode()[0])
df_test['BsmtFinType1'] = df_test['BsmtFinType1'].fillna(df_test['BsmtFinType1'].mode()[0])
df_test['BsmtFinSF1'] = df_test['BsmtFinSF1'].fillna(df_test['BsmtFinSF1'].mean())
df_test['BsmtFinType2'] = df_test['BsmtFinType2'].fillna(df_test['BsmtFinType2'].mode()[0])
df_test['BsmtFinSF2'] = df_test['BsmtFinSF2'].fillna(df_test['BsmtFinSF2'].mean())
df_test['BsmtUnfSF'] = df_test['BsmtUnfSF'].fillna(df_test['BsmtUnfSF'].mean())
df_test['TotalBsmtSF'] = df_test['TotalBsmtSF'].fillna(df_test['TotalBsmtSF'].mean())
df_test['BsmtFullBath'] = df_test['BsmtFullBath'].fillna(df_test['BsmtFullBath'].mode()[0])
df_test['BsmtHalfBath'] = df_test['BsmtHalfBath'].fillna(df_test['BsmtHalfBath'].mode()[0])
df_test['KitchenQual'] = df_test['KitchenQual'].fillna(df_test['KitchenQual'].mode()[0])
df_test['Functional'] = df_test['Functional'].fillna(df_test['Functional'].mode()[0])
df_test['GarageType'] = df_test['GarageType'].fillna(df_test['GarageType'].mode()[0])
df_test['GarageYrBlt'] = df_test['GarageYrBlt'].fillna(df_test['GarageYrBlt'].mode()[0])
df_test['GarageFinish'] = df_test['GarageFinish'].fillna(df_test['GarageFinish'].mode()[0])
df_test['GarageCars'] = df_test['GarageCars'].fillna(df_test['GarageCars'].mode()[0])
df_test['GarageArea'] = df_test['GarageArea'].fillna(df_test['GarageArea'].mode()[0])
df_test['GarageQual'] = df_test['GarageQual'].fillna(df_test['GarageQual'].mode()[0])
df_test['GarageCond'] = df_test['GarageCond'].fillna(df_test['GarageCond'].mode()[0])
df_test['SaleType'] = df_test['SaleType'].fillna(df_test['SaleType'].mode()[0])

In [ ]:
df_test = handle_nom(df_test, columns)

### Aplicação da Rede Neural

In [ ]:
# Removendo a coluna Id
df_test = df_test.drop('Id', axis=1)

In [ ]:
# Predição para o conjunto de teste
test_pred = nn_model.predict(df_test)

In [ ]:
# Transformando o vetor resultante em um dataframe
df_result = pd.DataFrame(test_pred, columns=['SalePrice'])

In [ ]:
df_result.to_csv(cwd, + '/dataset/price_predicted.csv')